In [1]:
%pip install optuna
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import cv2
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import ResNet50
from matplotlib import pyplot as plt
import optuna


Note: you may need to restart the kernel to use updated packages.


2025-04-14 13:12:28.784106: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-14 13:12:29.009560: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744603949.094881 1187337 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744603949.120826 1187337 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-14 13:12:29.333330: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
# データの読み込みと前処理
csv_path = '/mnt/c/Users/admin/Documents/testB/data/train.csv'
data_dir = '/mnt/c/Users/admin/Documents/testB/data/train/train'

df = pd.read_csv(csv_path)
df_filtered = df[df['gender_status'] != 5]

In [ ]:
x_data = []
y_data = []

for index, row in df_filtered.iterrows():
    img_path = os.path.join(data_dir, row['image'])
    img = cv2.imread(img_path)
    if img is not None:
        img = cv2.resize(img, (244, 244))
        x_data.append(img)
        y_data.append(row['gender_status'])
    else:
        print(f"画像が見つからないか、読み込めませんでした: {img_path}")

# 拡張パラメータを設定してImageDataGeneratorを初期化
datagen = ImageDataGenerator(
    rotation_range=360,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    #vertical_flip=True,
    fill_mode='nearest'
)

# x_dataとy_dataをnumpy配列に変換（まだ変換されていない場合）
x_data = np.array(x_data)
y_data = np.array(y_data)

# データジェネレータをトレーニングデータに適用
datagen.fit(x_data)

# ジェネレータを使用して画像を拡張
augmented_images = []
augmented_labels = []

for x, y in zip(x_data, y_data):
    x = x.reshape((1,) + x.shape)  # 画像を(1, height, width, channels)にリシェイプ
    i = 0
    for batch in datagen.flow(x, batch_size=1):
        augmented_images.append(batch[0])
        augmented_labels.append(y)
        i += 1
        if i >= 2:  # 画像ごとの拡張数
            break

# 拡張データをnumpy配列に変換
augmented_images = np.array(augmented_images)
augmented_labels = np.array(augmented_labels)

# 元のデータと拡張データを結合
x_data = np.concatenate((x_data, augmented_images), axis=0)
y_data = np.concatenate((y_data, augmented_labels), axis=0)

#クラスの不均等調整
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_data), y=y_data)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

#trainデータとtestデータをシャッフルしてクラスの均等性を揃える
x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.3, random_state=42, stratify=y_data)
x_train, y_train = shuffle(x_train, y_train, random_state=42)
x_val, y_val = shuffle(x_val, y_val, random_state=42)

y_train_categorical = to_categorical(y_train, num_classes=8)
y_val_categorical = to_categorical(y_val, num_classes=8)

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [4]:
#モデルの構築
def build_model(input_shape, learning_rate):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in base_model.layers:
        layer.trainable = False
    
    inputs = Input(shape=input_shape)
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.25)(x)
    outputs = Dense(8, activation='softmax')(x)
    
    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    return model
    

In [5]:
#モデルの訓練と評価の定義
def train_and_evaluate(model, x_train, y_train, x_val, y_val, class_weights, batch_size):
    history = model.fit( x_train, y_train, batch_size=batch_size
                        , epochs=300, validation_data=(x_val, y_val)
                        , class_weight=class_weights, verbose=0
                        , callbacks=[early_stopping]
                        , shuffle=True)
    _, accuracy = model.evaluate(x_val, y_val, verbose=0)
    return accuracy

In [6]:
#Optunaの目的関数の定義
def objective(trial):
    img_size = trial.suggest_categorical('img_size',[150,165])
    #img_height = trial.suggest_int('img_height', 32, 128)
    #img_width = trial.suggest_int('img_width', 32, 128)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3,log=True)
    #dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.5)
    #dense_units = trial.suggest_categorical('dense_units', [256, 512, 1024])
    #epochs = trial.suggest_int('epochs', 100, 300)
    batch_size = trial.suggest_categorical('batch_size', [128, 256, 512])
    #int型をカテゴリ型に全部変える、heitghtとwidthをsizeとして揃える、learning_rateの数値にlogをつけないといけない
    input_shape = (img_size, img_size, 3)
    model = build_model(input_shape, learning_rate,)

    # データのリサイズ
    x_train_resized = tf.image.resize(x_train, [img_size, img_size])
    x_val_resized = tf.image.resize(x_val, [img_size, img_size])
    
    accuracy = train_and_evaluate(model, x_train_resized, y_train_categorical, x_val_resized, y_val_categorical, class_weights_dict, batch_size)
    return accuracy

In [7]:
# Optunaのstudyを作成し、最適化を実行
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2025-04-14 13:14:55,742] A new study created in memory with name: no-name-6c084f7b-bf83-4a20-a000-9beba9e413c2
I0000 00:00:1744604096.268470 1187337 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46009 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:c3:00.0, compute capability: 8.6
I0000 00:00:1744604113.345484 1187862 service.cc:148] XLA service 0x7f1100002210 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1744604113.345797 1187862 service.cc:156]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
2025-04-14 13:15:13.572475: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1744604114.734214 1187862 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-04-14 13:15:16.330948: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:39

KeyboardInterrupt: 

In [ ]:
#結果の確認
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
#最適化されたハイパーパラメータの使用
best_params = study.best_trial.params
best_img_size = best_params['img_size']
#best_img_height = best_params['img_height']
#best_img_width = best_params['img_width']
best_learning_rate = best_params['learning_rate']
#best_dropout_rate = best_params['dropout_rate']
#best_dense_units = best_params['dense_units']
#best_epochs = best_params['epochs']
best_batch_size = best_params['batch_size']

#early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [ ]:
input_shape = (best_img_size, best_img_size, 3)
model = build_model(input_shape, best_learning_rate)

In [ ]:
x_train_resized = tf.image.resize(x_train, [best_img_size, best_img_size])
x_val_resized = tf.image.resize(x_val, [best_img_size, best_img_size])

In [ ]:
history = model.fit(x_train_resized
                    , y_train_categorical
                    , epochs=300
                    , batch_size=best_batch_size
                    , validation_data=(x_val_resized, y_val_categorical)
                    , class_weight=class_weights_dict
                    , callbacks=[early_stopping]
                    , shuffle=True)

In [ ]:
model.summary()

In [ ]:
# エポック数
epochs = range(1, len(history.history['loss']) + 1)

# 損失のプロット
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs, history.history['loss'], 'ro-', label='Training loss')
plt.plot(epochs, history.history['val_loss'], 'bo-', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# 精度のプロット
plt.subplot(1, 2, 2)
plt.plot(epochs, history.history['accuracy'], 'ro-', label='Training accuracy')
plt.plot(epochs, history.history['val_accuracy'], 'bo-', label='Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# プロットの表示
plt.tight_layout()
plt.show()

In [ ]:
# テストデータのディレクトリパスを設定
test_data_dir = '/mnt/c/Users/admin/Documents/testB/data/test/test'

# テストデータの画像を読み込み、前処理
x_test = []
image_names = []

for img_name in os.listdir(test_data_dir):
    img_path = os.path.join(test_data_dir, img_name)
    img = cv2.imread(img_path)
    if img is not None:
        img = cv2.resize(img, (best_img_size, best_img_size))
        x_test.append(img)
        image_names.append(img_name)
    else:
        print(f"画像が見つからないか、読み込めませんでした: {img_path}")

# NumPy配列に変換
x_test = np.array(x_test).reshape(-1, best_img_size, best_img_size, 3)

# 予測を行う
predictions = model.predict(x_test)
predicted_classes = np.argmax(predictions, axis=1)

# 結果をDataFrameにまとめる
results_df = pd.DataFrame({'image': image_names, 'gender_status': predicted_classes})

# 結果をCSVファイルに保存
results_csv_path = '/mnt/c/Users/admin/Documents/testB/data/predictions4.csv'
results_df.to_csv(results_csv_path, index=False)

print(f"予測結果が {results_csv_path} に保存されました。")